In [ ]:
import pandas as pd
import string
import re
from nltk.tokenize import word_tokenize

In [ ]:
df = pd.read_csv('', index_col='id')

In [ ]:
df['text'] = df['text'].apply(lambda x: x.lower())
def find_hashtags(text):
    return tuple(re.findall(r'#[\w]+', text))
def find_tagged(text):
    return tuple(re.findall(r'@[\w]+', text))
def find_urls(text):
    return tuple(re.findall(r'https?://(?:www\.)?\S+|www\.\S+', text))
df['hashtag'] = df['text'].apply(lambda x: find_hashtags(x))
df['tagged'] = df['text'].apply(lambda x: find_tagged(x))
df['url'] = df['text'].apply(lambda x: find_urls(x))
df['text_cleaned'] = df['text'].apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))
df['tokens'] = df['text_cleaned'].apply(lambda x: word_tokenize(x))

In [ ]:
def get_ngrams(tokens, n):
    return list([tokens[i: i + n] for i in range(len(tokens) - n)])

In [ ]:
df['bigrams'] = df['tokens'].apply(lambda x: get_ngrams(x, 2))
df['trigrams'] = df['tokens'].apply(lambda x: get_ngrams(x, 3))

In [ ]:
df_disaster = df[df['target'] == 1]
df_ordinary = df[df['target'] == 0]

In [ ]:
def get_top_values(dataframe, column, num_values=50):
    return pd.value_counts(
        list(sum(dataframe[column].values, ()))
    )[:num_values]
top_hashtags_disaster = get_top_values(df_disaster, 'hashtag')
top_hashtags_ordinary = get_top_values(df_ordinary, 'hashtag')

In [ ]:
df['has_disaster_hashtag'] = df['hashtag'].apply(lambda x: bool(sum([elt in top_hashtags_disaster for elt in x])))
df['has_ordinary_hashtag'] = df['hashtag'].apply(lambda x: bool(sum([elt in top_hashtags_ordinary for elt in x])))

In [ ]:
df